In [ ]:
!mkdir -p  ~/.kaggle
!cp kaggle.json  ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 98% 1.05G/1.06G [00:04<00:00, 208MB/s]
100% 1.06G/1.06G [00:04<00:00, 257MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout,Flatten,MaxPooling2D,Conv2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array,load_img,array_to_img,save_img
from keras.models import Sequential

In [ ]:
#generators
train_ds = keras.utils.image_dataset_from_directory(
    directory ='/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (150,150)
)


Found 20000 files belonging to 2 classes.


In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (150,150)

)

Found 5000 files belonging to 2 classes.


In [ ]:
conv_base = VGG16(
    weights = 'imagenet',
    input_shape  = (150,150,3),
    include_top = False

)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
#normalization

def process(image, label):
  image = tf.cast(image/255., tf.float32)
  return image, label


train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 16812353 (64.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
conv_base.trainable = False


In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs = 10, batch_size = 32, validation_data=(test_ds))

Epoch 1/10
625/625 [==============================] - 69s 97ms/step - loss: 0.2781 - accuracy: 0.8871 - val_loss: 0.2133 - val_accuracy: 0.9134
Epoch 2/10
625/625 [==============================] - 58s 92ms/step - loss: 0.1898 - accuracy: 0.9208 - val_loss: 0.2178 - val_accuracy: 0.9068
Epoch 3/10
625/625 [==============================] - 67s 106ms/step - loss: 0.1654 - accuracy: 0.9308 - val_loss: 0.2852 - val_accuracy: 0.8888
Epoch 4/10
625/625 [==============================] - 59s 93ms/step - loss: 0.1380 - accuracy: 0.9432 - val_loss: 0.2612 - val_accuracy: 0.9010
Epoch 5/10
625/625 [==============================] - 57s 91ms/step - loss: 0.1234 - accuracy: 0.9495 - val_loss: 0.2241 - val_accuracy: 0.9174
Epoch 6/10
625/625 [==============================] - 68s 108ms/step - loss: 0.0895 - accuracy: 0.9651 - val_loss: 0.3122 - val_accuracy: 0.8980
Epoch 7/10
625/625 [==============================] - 59s 94ms/step - loss: 0.0711 - accuracy: 0.9740 - val_loss: 0.2874 - val_accurac